In [10]:
import pandas as pd

In [11]:
df_games = pd.read_parquet('../Datasets/steam_games_preprocessed.parquet')
df_reviews = pd.read_parquet('../Datasets/user_reviews_preprocessed.parquet')

In [12]:
# Función para combinar géneros y etiquetas, eliminando duplicados.
def combine_features(row):
    genres_list = [genre.strip() for genre in row['genres'].split(',') if genre.strip()]
    tags_list = [tag.strip() for tag in row['tags'].split(',') if tag.strip()]
    combined_set = set(genres_list + tags_list)
    return ', '.join(sorted(combined_set))

df_games['features'] = df_games.apply(combine_features, axis=1)

In [13]:
df_games.head(1)

,item_id,title,developer,release_date,genres,tags,price,features
0,761140,Lost Summoner Kitty,Kotoshiro,2018-01-04,"Action, Casual, Indie, Simulation, Strategy","Strategy, Action, Indie, Casual, Simulation",4.99,"Action, Casual, Indie, Simulation, Strategy"


In [14]:
df_reviews.head(1)

,item_id,recommend,user_id,sentiment_analysis
0,1250,True,76561197970982479,2


In [15]:
# Merge de los dos dataframes con las columnas necesarias.

df_user_rec = pd.merge(df_games[['item_id', 'title', 'features']], df_reviews[['user_id','item_id','sentiment_analysis']], on='item_id')

In [16]:
df_user_rec.head(1)

,item_id,title,features,user_id,sentiment_analysis
0,282010,Carmageddon Max Pack,"1990's, Action, Classic, Gore, Indie, Multipla...",InstigatorAU,1


In [17]:
# Dropeamos item_id ya que no es necesario para el modelo

df_user_rec.drop(columns=['item_id'], inplace=True)

In [18]:
df_user_rec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48670 entries, 0 to 48669
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   title               48670 non-null  object
 1   features            48670 non-null  object
 2   user_id             48670 non-null  object
 3   sentiment_analysis  48670 non-null  int8  
dtypes: int8(1), object(3)
memory usage: 1.2+ MB


In [23]:
# Exportamos el dataframe a un archivo parquet comprimido por snappy.

df_user_rec.to_parquet('../Datasets/user_recommendations.parquet', compression='snappy', index=False)